In [2]:
#!/usr/bin/env python3

import sys
sys.path.insert(0, '../pyaf/py_aff3ct/build/lib')

import numpy as np
import py_aff3ct as aff3ct
import math
import time
import matplotlib.pyplot as plt
from datetime import timedelta
from tempfile import TemporaryFile

from py_aff3ct.module.py_module import Py_Module
import py_aff3ct.tools.frozenbits_generator as tool_fb
import py_aff3ct.tools.noise as tool_noise
import py_aff3ct.tools.sparse_matrix as tool_sp
import random 

class source(Py_Module):
	# 
	# out : r_in	-> initial vector : never modified
	#		enable	-> Always true
	#		delta_x -> From -4 to 4
	#		delta_y -> From -4 to 4
	#		ix_x	-> Constant for now
	#		ix_y	-> Constant for now
	#
	def generate(self, r_in, enable, delta_x, delta_y, ix_x, ix_y):
		for x in range(len(self.delta_x_range)):
			for y in range(len(self.delta_y_range)):
				delta_x[:] = self.delta_x_range[x] # Looks like : [[number]] with number = {-4 ; 3.95}
				delta_y[:] = self.delta_y_range[y] # Looks like : [[number]] with number = {-4 ; 3.95}
		return 0
    
	def __init__(self, r_in):

		Py_Module.__init__(self) 	# Call the aff3ct Py_Module __init__
		self.name   = "source_xy"   # Set your module's name
		self.r_in   = r_in			# Constant
		self.r_out  = r_in			# Sera modifié
		self.enable = np.ndarray(shape = (1,1),  dtype = np.int32  ) 
		self.ix_x   = 0 			# np.ndarray(shape = (1,1),  dtype = np.int32  )		# Position x dans le vecteur
		self.ix_y   = 1 			# np.ndarray(shape = (1,1),  dtype = np.int32  )		# Position y dans le vecteur
		self.enable[:] = 1			# Enable always true for now
		self.delta_x_range = np.arange(-4,4,0.05)
		self.delta_y_range = np.arange(-4,4,0.05)

		t_generate = self.create_task("generate") # create a task for your module

		s_r_in    = self.create_socket_out  (t_generate, "r_in"   , len(r_in), np.float32 ) # create an output socket for the task t_generate
		s_enable  = self.create_socket_out  (t_generate, "enable" , 1		 , np.int32   ) # create an output socket for the task t_generate
		s_delta_x = self.create_socket_out  (t_generate, "delta_x", 1		 , np.float32 )	# create an output socket for the task t_generate
		s_delta_y = self.create_socket_out  (t_generate, "delta_y", 1	 	 , np.float32 ) # create an output socket for the task t_generate
		s_ix_x    = self.create_socket_out  (t_generate, "ix_x"   , 1		 , np.int32   ) # create an output socket for the task t_generate
		s_ix_y    = self.create_socket_out  (t_generate, "ix_y"   , 1		 , np.int32   ) # create an output socket for the task t_generate

		self.create_codelet(t_generate, lambda slf, lsk, fid: slf.add(lsk[s_r_in], lsk[s_enable], lsk[s_delta_x], lsk[s_delta_y], lsk[s_ix_x], lsk[s_ix_y])) # create codelet